# 1 Prepear Work

In [1]:
import os.path, jieba

In [2]:
import pandas as pd

In [3]:
Word2vec_model_path = r'D:\Assignment\Project_01\Word2vec_model.w2v'

In [4]:
related_word_path = r'D:\Assignment\Project_01\related_word.txt'

In [5]:
import gensim

D:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [6]:
word2vec = gensim.models.word2vec.Word2Vec.load(Word2vec_model_path).wv

In [7]:
key = word2vec.vocab.keys()

In [8]:
list(key)[:10]

['此外', '自', '本周', '6', '月', '12', '日起', '除', '小米', '手机']

In [9]:
word2vec.most_similar('体验版', topn=10)

[('稳定版', 0.9151314496994019),
 ('乐东黎族自治县', 0.9127017259597778),
 ('御史大夫', 0.91154944896698),
 ('范丽青', 0.9077432155609131),
 ('饰件', 0.9041963815689087),
 ('陈瑞谢', 0.9034069776535034),
 ('酥碱', 0.9024229049682617),
 ('北京火车站', 0.902195155620575),
 ('长缆', 0.9020404815673828),
 ('诺氟沙星', 0.8997544050216675)]

In [10]:
def get_related_word(file_path):
    file = open(file_path, "r", encoding="utf-8")
    content_line = file.readline()
    result = []
    i = 0
    while content_line:
        temp = ''
        content_line = content_line.strip("\n")
        if len(content_line) > 0:
            result.append(content_line)
        content_line = file.readline()

    return result

In [11]:
related_word = get_related_word(related_word_path)

In [12]:
related_word[:10]

['说', '指出', '表示', '认为', '坦言', '透露', '看来', '告诉', '提到', '所说']

# 2 SIF

## 2.1 Get the probality of word

In [13]:
from collections import Counter

In [14]:
def get_probability(word, model):
    keys = model.vocab.keys()
    total_words_count = sum([v.count for k, v in model.vocab.items()])
    esp = 1 / total_words_count

    if word in keys:
        word_count = model.vocab[word].count
        return word_count / total_words_count
    else:
        return esp

In [15]:
get_probability('啊', word2vec)

4.5933924963048614e-05

## 2.2 Sentences embdding

In [16]:
import numpy as np

In [17]:
from sklearn.decomposition import PCA

In [18]:
def get_SIF(word, model, a=0.01):
    return (a + get_probability(word, model))

In [19]:
def SIF_sentence_embdding(sentence, model, a=0.01):
    result = 0
    i = 0
    for word in sentence:
        temp = model[word] * get_SIF(word, model)
        result += temp
    return result

In [20]:
def PCA_SIF_sentence_embdding(news, model=word2vec, embedding_size=35, a=0.01):
    X = [SIF_sentence_embdding(sentence, model) for sentence in news]
    
    pca = PCA(n_components=min(embedding_size, len(X)))
    
    pca.fit(np.array(X))
    
    u = pca.components_[0]  # the PCA vector
    u = np.multiply(u, np.transpose(u))  # u x uT
    
    # pad the vector?  (occurs if we have less sentences than embeddings_size)
    if len(u) < embedding_size:
        for i in range(embedding_size - len(u)):
            u = np.append(u, 0)  # add needed extension for multiplication below
    # resulting sentence vectors, vs = vs -u x uT x vs
    Y = []
    for Vs in X:
        sub = np.multiply(u, Vs)
        Y.append(np.subtract(Vs, sub))
    return Y

In [21]:
from collections import defaultdict

In [22]:
def generate_all_PCA_SIF_sentence_embdding(content):
    result = []
    for news in content:
        yield PCA_SIF_sentence_embdding(news)

In [23]:
#sentence_embdding = generate_all_PCA_SIF_sentence_embdding(sentence_word_content)

In [24]:
from scipy.spatial.distance import cosine

In [25]:
def distance(v1, v2):
    return cosine(v1, v2)

# 3 NER & Dependency Prasing

In [26]:
from stanfordcorenlp import StanfordCoreNLP

In [27]:
nlp = StanfordCoreNLP(r'D:\stanford_nlp', lang='zh')

# 4 execute program

In [28]:
import re, jieba

In [29]:
def token(string):
    return ''.join(re.findall(r'[\d|\w]+', string))
def cut(string):
    return ' '.join(jieba.cut(string))
def cut_sentence(para):
    para = re.sub('([。！？\?])([^”’])', r"\1\n\2", para)  # 单字符断句符
    para = re.sub('(\.{6})([^”’])', r"\1\n\2", para)  # 英文省略号
    para = re.sub('(\…{2})([^”’])', r"\1\n\2", para)  # 中文省略号
    para = re.sub('([。！？\?][”’])([^，。！？\?])', r'\1\n\2', para)
    # 如果双引号前有终止符，那么双引号才是句子的终点，把分句符\n放到双引号后，注意前面的几句都小心保留了双引号
    para = para.rstrip()  # 段尾如果有多余的\n就去掉它
    # 很多规则中会考虑分号;，但是这里我把它忽略不计，破折号、英文双引号等同样忽略，需要的再做些简单调整即可。
    return para.split("\n")

In [30]:
def get_sentence_content_with_punctuation(news):
    news = re.sub(r'\n', '', news)
    if news != '' and news != ' ':
        result = cut_sentence(news)
    return result

In [31]:
def get_sentence_content(news_sentence_with_punctuation):
    result = []
    
    for sentence in news_sentence_with_punctuation:
        temp = token(sentence)
        result.append(temp)
    return result

In [32]:
def get_sentence_word_content(news_sentence):
    result = []
    
    for sentence in news_sentence:
        temp_word = jieba.lcut(sentence)
        result.append(temp_word)
    return result

In [33]:
def memo(func):
    cache = {}

    def _wrap(*args): ## ? *args, **kwargs
        if args in cache: result = cache[args]
        else:
            result = func(*args)
            cache[args] = result
        return result
    return _wrap

In [34]:
def NLP_NER(sentence):
    return nlp.ner(sentence)

In [35]:
def NLP_POS(sentence):
    return nlp.pos_tag(sentence)

In [36]:
def NLP_DPR(sentence):
    return nlp.dependency_parse(sentence)

In [37]:
def SUBJ(string):
    return re.search('subj', string)

In [38]:
@memo
def judge_pronoun(sentence):
    subsentence = re.search('(.+)“|”(.+)', sentence)
    if tmp_sen:
        sentence = subsentence.group(1)
    POS = NLP_POS(sentence)
    D_pr = NLP_DPR(sentence)
    for relation in D_pr:
        i = relation[-1] - 1
        if SUBJ(relation[0]) and (POS[i][-1] == 'PN' or POS[i][-1] == 'PRP'):
            return True
    return False

In [39]:
def get_name(subj_index, word_cut, D_pr):
    name = word_cut[subj_index]
    for relation in D_pr:
        if relation[0] == 'appos':continue
        if relation[1] - 1 == subj_index:
            name = word_cut[relation[2] - 1] + name
            print(relation)
            print(word_cut[relation[2] - 1])
    return name

In [40]:
def get_saying(predicate_index, word_cut, D_pr):
    if ':' in word_cut:
        return ''.join(word_cut[word_cut.index(':')+1:])
    
    return ''.join(word_cut[predicate_index+1:])

In [41]:
def parse_sentence(sentence):
    name = ''
    saying = ''
    NER = NLP_NER(sentence)
    word_cut = [w for w, _ in NER]
    
    #寻找句中第一个与‘说’相关的词汇，若无则直接停止解析，返回False。
    say_related_word = [word for word in word_cut if word in related_word]
    if not say_related_word: return False
    
    D_pr = NLP_DPR(sentence)
    
    for i, relation in enumerate(D_pr):
        
        #首先进行句式解析，判断依赖‘说’的主语是否为'PERSON/ ORGANIZATION/ LOCATION'。
        d = relation[1] - 1 #谓语索引
        k = relation[2] - 1 #主语索引
        
        if (word_cut[d] in say_related_word) and SUBJ(relation[0]):  #找出第一个主谓结构
             if (NER[k][-1] == 'PERSON' or NER[k][-1] == 'ORGANIZATION' or NER[k][-1] == 'LOACTION'):
                name = get_name(k, word_cut, D_pr)
                saying = get_saying(d, word_cut, D_pr)
                
                if not saying:
                    quotations = re.findall(r'“(.+?)”', sentence)
                    if quotations: 
                        saying = quotations[-1]
                return name, saying
            
        #若句子中有与‘说’相关词汇，且存在‘：’，则直取其后言论。
        if word_cut[i][0] == ':':
            for j in range(j):
                if NER[j][-1] == 'PERSON' or NER[j][-1] == 'ORGANIZATION' or NER[j][-1] == 'LOACTION':
                    name += NER[j][-1]
            saying = ''.join(word_cut[i+1:])
            return name, saying
    return False

In [42]:
def compare_sentence(sen1_index, sen2_index, Y):
    return cosine(Y[sen1_index], Y[sen2_index])

In [43]:
def get_person_and_point(input_text):
    res = []
    V_point = defaultdict(list)
    flag = -1
    
    news_sentence_with_punctuation = get_sentence_content_with_punctuation(input_text)
    news_sentence = get_sentence_content(news_sentence_with_punctuation)
    
    news_word_with_punctuation = get_sentence_word_content(news_sentence_with_punctuation)
    news_word = get_sentence_word_content(news_sentence)
    #print(news_sentence_with_punctuation)
    #print(news_sentence)
    #print(news_word_with_punctuation)
    #print(news_word)
    Y = PCA_SIF_sentence_embdding(news_word)
    
    for j, sentence in enumerate(news_sentence_with_punctuation):
        expect = 0.75 #sentence embdding比较因素
        print(sentence)
        
        tmp_sentence_index = "" #储存前一个言论的索引
        tmp_res = []
        #特殊情况处理：当句子中第一个字符出现“
        if sentence[0] == '“':
            the_subsen_of_people_in == re.search('”(.+)“|”(.+)', sentence)
            if the_subsen_of_people_in:
                the_subsen_of_people_in = [sen for sen in the_subsen_of_people_in.groups() if sen][0]
                
                saying = sentence.replace(the_subsen_of_people_in, '') #剩余部分即为言论
                if res and judge_pronoun(the_subsen_of_people_in):
                    res[-1][1] += saying
                else:
                    tmp_res = parse_sentence(the_subsen_of_people_in)
                    if tmp_res:
                        saying += tmp_res[1] if tmp_res else ''
                        res.append(tmp_res[0], saying)
                continue
            elif res:
                res[-1][1] += sentence
                continue
            else:
                continue
        
        #一般情况处理
        #提取：发言人，言论内容
        tmp_res = parse_sentence(sentence)
        
        #一般情况处理（1）：不存在发言人
        if not tmp_res:
            if res and tmp_sentence_index and compare_sentence(tmp_sentence_index, j, Y) > expect:
                res[-1][1] += sentence
                tmp_sentence_index = j
            continue
            
        #一般情况处理（2）：存在发言人
        if tmp_res[-1]:
            res.append(tmp_res)
        tmp_sentence_index = j
    return res
        

In [44]:
def get_person_and_point_dict(input_text):
    res = defaultdict(list)
    
    tmp = get_person_and_point(input_text)
    
    for person, saying in tmp:
        if saying[0] in ['，', '。', '、', '）', '!', '?']:
            saying = saying[1:]
        res[person].append(saying)
    return res

In [45]:
if '认为' in related_word:
    print('y')
else:
    print('n')

y


In [51]:
test = '\u3000\u3000原标题：叙利亚被“袭机”事件惹怒俄罗斯 警告将瞄准美战机\n\u3000\u3000海外网6月19日电 当地时间6月19日，俄罗斯国防部对美国军方击落叙利亚飞机一事作出反击，宣布停止执行俄美两国签署的“在叙飞行安全备忘录”，并称以后美国领导的国际联军所有的战机，都是俄罗斯军方监控与瞄准的目标，叙利亚局势进一步复杂化。\n\u3000\u3000据纽约时报消息，由于美国军方今日击落了一架叙利亚军机，俄罗斯国防部发布消息，自6月19日起暂停执行俄美间在叙利亚领空“防止空中事件和保障行动期间飞行安全”的相互谅解备忘录。要求美方指挥部对此事件进行彻查，结果与俄方共享。\n\u3000\u3000公告称：“俄空军在叙利亚领空执行任务的地区里，幼发拉底河西岸发现的任何飞行物，包括美国领导的国际联军的飞机和无人机，都将是俄罗斯军方地面和空中防空武器监控与瞄准的目标。”\n\u3000\u3000据叙利亚军方声明，当地时间6月19日，一架政府军机正前往拉卡（Raqqa）市，准备对盘踞于此的IS武装分子进行打击，却突然遭到美军袭击，飞行员至今失踪。声明称：“这次袭击发生的时机，是在叙利亚政府及其盟国的军队在与IS恐怖分子的战斗中获得优势的情况下发生的，本来这些恐怖分子已经在叙利亚的沙漠中节节败退。”\n\u3000\u3000此次“袭机”事件“惹怒”了俄罗斯，俄罗斯参议院国防委员会副主席弗朗茨·克莱琴谢夫（Frants Klintsevich）称美军的行动是“挑衅行为”，实际上是对叙利亚的“军事侵略”。\n\u3000\u3000该部门认为，美军“故意不履行双方2015年签署的“安全备忘录”中规定的义务，因此宣布暂停与美军在该框架下的合作。据报道，该协议一直是美俄两国军队协调在该地区的军事活动的关键。俄罗斯、美国、叙利亚、土耳其等国家在叙利亚的诉求经常是相冲突的，该协议就在其中起到调和作用。在特朗普四月下令袭击叙利亚空军之后，俄方表示将暂停协议，不过几个星期之后又重启，这次时隔两月后再被中断。\n\u3000\u3000俄罗斯外长拉夫罗夫在回应记者时也表示，“涉及到叙利亚地面所发生的事情，毫无疑问，我们认为有必要尊重叙利亚的主权和领土完整，这是联合国2254号决议和其他文件规定的。因此，任何地面行动，包括实施军事行动的参与方，需得到大马士革的许可。”（编译/海外网 杨佳）\n'

In [54]:
V_point = get_person_and_point_dict(test)

　　原标题：叙利亚被“袭机”事件惹怒俄罗斯 警告将瞄准美战机　　海外网6月19日电 当地时间6月19日，俄罗斯国防部对美国军方击落叙利亚飞机一事作出反击，宣布停止执行俄美两国签署的“在叙飞行安全备忘录”，并称以后美国领导的国际联军所有的战机，都是俄罗斯军方监控与瞄准的目标，叙利亚局势进一步复杂化。
　　据纽约时报消息，由于美国军方今日击落了一架叙利亚军机，俄罗斯国防部发布消息，自6月19日起暂停执行俄美间在叙利亚领空“防止空中事件和保障行动期间飞行安全”的相互谅解备忘录。
要求美方指挥部对此事件进行彻查，结果与俄方共享。
　　公告称：“俄空军在叙利亚领空执行任务的地区里，幼发拉底河西岸发现的任何飞行物，包括美国领导的国际联军的飞机和无人机，都将是俄罗斯军方地面和空中防空武器监控与瞄准的目标。”
　　据叙利亚军方声明，当地时间6月19日，一架政府军机正前往拉卡（Raqqa）市，准备对盘踞于此的IS武装分子进行打击，却突然遭到美军袭击，飞行员至今失踪。
声明称：“这次袭击发生的时机，是在叙利亚政府及其盟国的军队在与IS恐怖分子的战斗中获得优势的情况下发生的，本来这些恐怖分子已经在叙利亚的沙漠中节节败退。”
　　此次“袭机”事件“惹怒”了俄罗斯，俄罗斯参议院国防委员会副主席弗朗茨·克莱琴谢夫（Frants Klintsevich）称美军的行动是“挑衅行为”，实际上是对叙利亚的“军事侵略”。
('parataxis:prnmod', 18, 21)
Klintsevich
　　该部门认为，美军“故意不履行双方2015年签署的“安全备忘录”中规定的义务，因此宣布暂停与美军在该框架下的合作。
据报道，该协议一直是美俄两国军队协调在该地区的军事活动的关键。
俄罗斯、美国、叙利亚、土耳其等国家在叙利亚的诉求经常是相冲突的，该协议就在其中起到调和作用。
在特朗普四月下令袭击叙利亚空军之后，俄方表示将暂停协议，不过几个星期之后又重启，这次时隔两月后再被中断。
　　俄罗斯外长拉夫罗夫在回应记者时也表示，“涉及到叙利亚地面所发生的事情，毫无疑问，我们认为有必要尊重叙利亚的主权和领土完整，这是联合国2254号决议和其他文件规定的。
因此，任何地面行动，包括实施军事行动的参与方，需得到大马士革的许可。”
（编译/海外网 杨佳）


In [55]:
V_point

defaultdict(list,
            {'Klintsevich弗朗茨·克莱琴谢夫': ['美军的行动是“挑衅行为”，实际上是对叙利亚的“军事侵略”。'],
             '罗夫': ['“涉及到叙利亚地面所发生的事情，毫无疑问，我们认为有必要尊重叙利亚的主权和领土完整，这是联合国2254号决议和其他文件规定的。']})